# PDF 내 이미지 추출해 활용

In [63]:
#https://github.com/teddylee777/langchain-kr/blob/main/12-RAG/10-Multi_modal_RAG-GPT-4o.ipynb

In [15]:
#! pip install -U langchain openai chromadb langchain-experimental

In [16]:
#! pip install "unstructured[all-docs]" pillow pydantic lxml pillow matplotlib chromadb tiktoken

In [1]:
# 파일 경로
fpath = "D:/[24]ICT_Practice/practice_file"
fname = "login.pdf"

In [2]:
import os
from langchain_text_splitters import CharacterTextSplitter
from unstructured.partition.pdf import partition_pdf

In [3]:
#PDF에서 요소 추출
def extract_pdf_elements(path, fname):
    
    #PDF 파일에서 이미지, 테이블, 그리고 텍스트 조각을 추출합니다.
    #path: 이미지(.jpg)를 저장할 파일 경로
    #fname: 파일 이름
    
    return partition_pdf(
        filename=os.path.join(path, fname),
        extract_images_in_pdf=True,  # PDF 내 이미지 추출 활성화
        infer_table_structure=True,  # 테이블 구조 추론 활성화
        chunking_strategy="by_title",  # 제목별로 텍스트 조각화
        max_characters=4000,  # 최대 문자 수
        new_after_n_chars=3800,  # 이 문자 수 이후에 새로운 조각 생성
        combine_text_under_n_chars=2000,  # 이 문자 수 이하의 텍스트는 결합
        image_output_dir_path=path,  # 이미지 출력 디렉토리 경로
        poppler_path=r"D:\poppler-24.02.0\Library\bin", # 추가: poppler 경로 명시적 지정
        tesseract_path=r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # 추가: tesseract 경로 명시적 지정
    )

In [4]:
# 요소를 유형별로 분류


def categorize_elements(raw_pdf_elements):

    # PDF에서 추출된 요소를 테이블과 텍스트로 분류합니다.
    # raw_pdf_elements: unstructured.documents.elements의 리스트

    tables = []  # 테이블 저장 리스트
    texts = []  # 텍스트 저장 리스트
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))  # 테이블 요소 추가
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            texts.append(str(element))  # 텍스트 요소 추가
    return texts, tables

In [5]:
# 요소 추출
raw_pdf_elements = extract_pdf_elements(fpath, fname)

# 텍스트, 테이블 추출
texts, tables = categorize_elements(raw_pdf_elements)

# 선택사항: 텍스트에 대해 특정 토큰 크기 적용
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=4000, chunk_overlap=0  # 텍스트를 4000 토큰 크기로 분할, 중복 없음
)
joined_texts = " ".join(texts)  # 텍스트 결합
texts_4k_token = text_splitter.split_text(joined_texts)  # 분할 실행

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
len(texts_4k_token)

2

## 텍스트 및 테이블 요약

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [13]:
# 텍스트 요소의 요약 생성


def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    텍스트 요소 요약
    texts: 문자열 리스트
    tables: 문자열 리스트
    summarize_texts: 텍스트 요약 여부를 결정. True/False
    """

    # 프롬프트 설정
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    # 텍스트 요약 체인
    model = ChatOpenAI(temperature=0, model="gpt-4", openai_api_key="개인키작성")
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # 요약을 위한 빈 리스트 초기화
    text_summaries = []
    table_summaries = []

    # 제공된 텍스트에 대해 요약이 요청되었을 경우 적용
    if texts and summarize_texts:
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
    elif texts:
        text_summaries = texts

    # 제공된 테이블에 적용
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

    return text_summaries, table_summaries

In [15]:
# 텍스트, 테이블 요약 가져오기
text_summaries, table_summaries = generate_text_summaries(
    texts_4k_token, tables, summarize_texts=True
)

## 이미지 요약

In [16]:
import base64
import os

from langchain_core.messages import HumanMessage

In [17]:
def encode_image(image_path):
    # 이미지 파일을 base64 문자열로 인코딩합니다.
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(img_base64, prompt):
    # 이미지 요약을 생성합니다.
    chat = ChatOpenAI(model="gpt-4o", max_tokens=2048)

    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content

In [18]:
def generate_img_summaries(path):
    """
    이미지에 대한 요약과 base64 인코딩된 문자열을 생성합니다.
    path: Unstructured에 의해 추출된 .jpg 파일 목록의 경로
    """

    # base64로 인코딩된 이미지를 저장할 리스트
    img_base64_list = []

    # 이미지 요약을 저장할 리스트
    image_summaries = []

    # 요약을 위한 프롬프트
    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval."""

    # 이미지에 적용
    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path, img_file)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image, prompt))

    return img_base64_list, image_summaries


In [19]:
# 이미지 요약 실행
img_base64_list, image_summaries = generate_img_summaries(fpath)

In [20]:
len(image_summaries)

0

## 벡터 저장소에 추가하기

원본 문서와 문서 요약을 Multi Vector Retriever에 추가

In [21]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

In [22]:
def create_multi_vector_retriever(
    vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images
):
    """
    요약을 색인화하지만 원본 이미지나 텍스트를 반환하는 검색기를 생성합니다.
    """

    # 저장 계층 초기화
    store = InMemoryStore()
    id_key = "doc_id"

    # 멀티 벡터 검색기 생성
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # 문서를 벡터 저장소와 문서 저장소에 추가하는 헬퍼 함수
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [
            str(uuid.uuid4()) for _ in doc_contents
        ]  # 문서 내용마다 고유 ID 생성
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]
        retriever.vectorstore.add_documents(
            summary_docs
        )  # 요약 문서를 벡터 저장소에 추가
        retriever.docstore.mset(
            list(zip(doc_ids, doc_contents))
        )  # 문서 내용을 문서 저장소에 추가

    # 텍스트, 테이블, 이미지 추가
    if text_summaries:
        add_documents(retriever, text_summaries, texts)

    if table_summaries:
        add_documents(retriever, table_summaries, tables)

    if image_summaries:
        add_documents(retriever, image_summaries, images)

    return retriever

In [25]:
# 요약을 색인화하기 위해 사용할 벡터 저장소
vectorstore = Chroma(
    collection_name="sample-rag-multi-modal", embedding_function=OpenAIEmbeddings(openai_api_key="개인키작성")
)

# 검색기 생성
retriever_multi_vector_img = create_multi_vector_retriever(
    vectorstore,
    text_summaries,
    texts,
    table_summaries,
    tables,
    image_summaries,
    img_base64_list,
)

## RAG（검색기 구축）

In [26]:
import io
import re

from IPython.display import HTML, display
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from PIL import Image

In [28]:
def plt_img_base64(img_base64):
    """base64 인코딩된 문자열을 이미지로 표시"""
    # base64 문자열을 소스로 사용하는 HTML img 태그 생성
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    # HTML을 렌더링하여 이미지 표시
    display(HTML(image_html))


def looks_like_base64(sb):
    """문자열이 base64로 보이는지 확인"""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None

In [29]:
def is_image_data(b64data):
    """
    base64 데이터가 이미지인지 시작 부분을 보고 확인
    """
    image_signatures = {
        b"\xff\xd8\xff": "jpg",
        b"\x89\x50\x4e\x47\x0d\x0a\x1a\x0a": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # 처음 8바이트를 디코드하여 가져옴
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False

In [30]:
def resize_base64_image(base64_string, size=(128, 128)):
    """
    Base64 문자열로 인코딩된 이미지의 크기 조정
    """
    # Base64 문자열 디코드
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # 이미지 크기 조정
    resized_img = img.resize(size, Image.LANCZOS)

    # 조정된 이미지를 바이트 버퍼에 저장
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # 조정된 이미지를 Base64로 인코딩
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

In [31]:
def split_image_text_types(docs):
    """
    base64로 인코딩된 이미지와 텍스트 분리
    """
    b64_images = []
    texts = []
    for doc in docs:
        # 문서가 Document 타입인 경우 page_content 추출
        if isinstance(doc, Document):
            doc = doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            doc = resize_base64_image(doc, size=(1300, 600))
            b64_images.append(doc)
        else:
            texts.append(doc)
    return {"images": b64_images, "texts": texts}

In [32]:
def img_prompt_func(data_dict):
    """
    컨텍스트를 단일 문자열로 결합
    """
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # 이미지가 있으면 메시지에 추가
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)

    # 분석을 위한 텍스트 추가
    text_message = {
        "type": "text",
        "text": (
            "You are financial analyst tasking with providing investment advice.\n"
            "You will be given a mixed of text, tables, and image(s) usually of charts or graphs.\n"
            "Use this information to provide investment advice related to the user question. Answer in Korean. Do NOT translate company names.\n"
            f"User-provided question: {data_dict['question']}\n\n"
            "Text and / or tables:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)
    return [HumanMessage(content=messages)]

In [61]:
def multi_modal_rag_chain(retriever):
    """
    멀티모달 RAG 체인
    """

    # 멀티모달 LLM
    model = ChatOpenAI(temperature=0, model="gpt-4o", max_tokens=2048, 
                       openai_api_key="개인키작성", n_results=3)

    # RAG 파이프라인
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | model
        | StrOutputParser()
    )

    return chain

## 검사

우리가 질문에 관련된 이미지를 검색할 때, 관련성 있는 이미지들을 되돌려 받기

In [57]:
# 검색 질의 실행
query = "회원가입 어떻게 해?"

# 질의에 대한 문서 6개를 검색합니다.
docs = retriever_multi_vector_img.invoke(query, limit=6)

# 문서의 개수 확인
len(docs)  # 검색된 문서의 개수를 반환합니다.

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


3

In [58]:
# 관련 이미지를 반환
plt_img_base64(docs[0])

아마 노트북 문제로 이미지 안 뜸,,

## RAG

In [62]:
# RAG 체인 실행
query = "선박제원 신고는 어떻게 해?"
print(chain_multimodal_rag.invoke(query))

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


선박제원 신고는 다음과 같은 절차를 통해 진행할 수 있습니다:

1. **PORT-MIS 웹사이트 접속**: 먼저 PORT-MIS 웹사이트(portmis.go.kr)에 접속합니다.

2. **로그인**: 회원가입이 되어 있지 않다면 회원가입을 진행하고, 이미 회원이라면 로그인합니다.

3. **신고 메뉴 선택**: 로그인 후, 상단 메뉴에서 '선박제원 신고' 또는 관련 메뉴를 선택합니다.

4. **신고서 작성**: 제공된 양식에 따라 선박의 제원 정보를 입력합니다. 여기에는 선박의 이름, 등록번호, 톤수, 길이, 폭, 깊이 등의 정보가 포함됩니다.

5. **첨부 파일 업로드**: 필요한 경우, 선박의 제원과 관련된 서류나 증빙 자료를 스캔하여 첨부 파일로 업로드합니다.

6. **검토 및 제출**: 입력한 정보를 다시 한번 검토한 후, 제출 버튼을 눌러 신고를 완료합니다.

7. **확인 및 문의**: 신고가 정상적으로 접수되었는지 확인하고, 추가 문의사항이 있을 경우 PORT-MIS 고객센터(전화번호: 010-1111-1111 또는 이메일: portmis@portmis.com)로 연락합니다.

이 절차를 통해 선박제원 신고를 완료할 수 있습니다. 추가적인 도움이 필요하면 PORT-MIS 고객센터에 문의하시기 바랍니다.


# pdf 데이터 활용

In [62]:
#https://wikidocs.net/234009

In [1]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [36]:
#문서 로드

In [4]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("D:/[24]ICT_Practice/practice_file/ex1.pdf")

# 페이지 별 문서 로드
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[0].page_content[:500]}")
print(f"\n[metadata]\n{docs[0].metadata}\n")

문서의 수: 1

[페이지내용]
방사능 오염  선박평형수 유입 차단 조치 안내
원전 오염수 해양 방류(’23.8.24.)에 따라 방사능 오염 선박평형수가 국내에
유입되지 않도록 아래와 같이 조치를 시행 하오니 협조하여 주시기 바랍니다.
선박평형수 방사능 오염조사 등 절차
일본 6개현
평형수 주입배출 예정 선박
미배출 예정 선박평형수 교환교환여부 검증
및 방사능 조사평형수 배출입항
입항
(평형수 배출금지)자료 제출(출항 1시간 전까지)
및 확인출항조치사항일본 동부 6개현* 에서 평형수를 주입 후 국내 입항 예정인 선박
국내 입항 24시간 전까지 평형수 입항보고서* 제출* 아오모리현, 이와테현, 후쿠시마현, 미야기현, 이바라기현, 치바현1
조치사항일본 동부 6개현에서 평형수를 주입 후 국내 배출 예정인 선박
우리나라 관할수역 밖* 에서 평형수 교환 후 입항2
* 쓰가루해협 등 일본 북쪽항로로 입항시 쓰가루해협 통과 후 부터 동경 135도30분 사이 또는
  간몬해협 등 일본 남쪽항로로 입항시 북위 34도35분 통과 후

[metadata]
{'source': 'D:/[24]ICT_Practice/practice_file/ex1.pdf', 'page': 0}



In [2]:
# 텍스트 분할 

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100, # 원하는 덩어리 사이즈
    chunk_overlap  = 20, # 겹치는 정도 (컨텍스트 유지 목적)
    length_function = len, # 길이 측정 함수
    add_start_index = True # 원본의 덩어리 시작 위치를 보여줄지 여부
)

In [6]:
from langchain.document_loaders import OnlinePDFLoader
data = loader.load()
texts = text_splitter.split_documents(data)
print(texts[10])
print(texts[11])

page_content='지방해양수산청에서는 출항 전까지 제출된 자료를 검토하여 미배출 여부를 확인' metadata={'source': 'D:/[24]ICT_Practice/practice_file/ex1.pdf', 'page': 0, 'start_index': 704}
page_content='(필요시 승선조사)할 예정이며, 선박의 출항은 배출 여부 확인 후 가능지방해양수산청에서 승선하여 교환 여부를 확인하고, 평형수 시료를 채취하여 신속하게방사능 농도를 측정할' metadata={'source': 'D:/[24]ICT_Practice/practice_file/ex1.pdf', 'page': 0, 'start_index': 746}


In [56]:
# 임베딩 & 벡터스토어 생성(Create Vectorstore)

In [53]:
#! pip install faiss-cpu

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings

# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(
    documents=texts, embedding=OpenAIEmbeddings(openai_api_key="개인키작성"))

In [55]:
# Retriever 생성

In [10]:
## way1: 코사인 유사도

In [ ]:
query = "일본 동부에서 평형수를 주입 후 국내 입항 예정인 선박은?"

retriever = vectorstore.as_retriever(search_type="similarity")
search_result = retriever.get_relevant_documents(query)
print(search_result)

C:\ProgramData\Anaconda3\envs\langchainenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


**커널 죽음 -> 왜?**

In [11]:
## way2: similarity_score_threshold (유사도 기반 검색에서 score_threshold 이상인 결과만 반환)

In [8]:
query = "일본 동부에서 평형수를 주입 후 국내 입항 예정인 선박은?"

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.8}
)
search_result = retriever.get_relevant_documents(query)
print(search_result)

C:\ProgramData\Anaconda3\envs\langchainenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'source': 'D:/[24]ICT_Practice/practice_file/ex1.pdf', 'page': 0, 'start_index': 342}, page_content='조치사항일본 동부 6개현에서 평형수를 주입 후 국내 배출 예정인 선박\n우리나라 관할수역 밖* 에서 평형수 교환 후 입항2'), Document(metadata={'source': 'D:/[24]ICT_Practice/practice_file/ex1.pdf', 'page': 0, 'start_index': 227}, page_content='및 확인출항조치사항일본 동부 6개현* 에서 평형수를 주입 후 국내 입항 예정인 선박'), Document(metadata={'source': 'D:/[24]ICT_Practice/practice_file/ex1.pdf', 'page': 0, 'start_index': 523}, page_content='조치사항일본 동부 6개현에서 평형수를 주입 후 국내 배출 계획이 없는 선박\n출항 1시간 전까지 평형수를 배출하지 않았음을 입증할 수 있는 자료* 제출3'), Document(metadata={'source': 'D:/[24]ICT_Practice/practice_file/ex1.pdf', 'page': 0, 'start_index': 72}, page_content='유입되지 않도록 아래와 같이 조치를 시행 하오니 협조하여 주시기 바랍니다.\n선박평형수 방사능 오염조사 등 절차\n일본 6개현\n평형수 주입배출 예정 선박')]


In [12]:
## way3: maximum marginal search result 를 사용하여 검색

In [13]:
query = "일본 동부에서 평형수를 주입 후 국내 입항 예정인 선박은?"

retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 2})
search_result = retriever.get_relevant_documents(query)
print(search_result)

[Document(metadata={'source': 'D:/[24]ICT_Practice/practice_file/ex1.pdf', 'page': 0, 'start_index': 342}, page_content='조치사항일본 동부 6개현에서 평형수를 주입 후 국내 배출 예정인 선박\n우리나라 관할수역 밖* 에서 평형수 교환 후 입항2'), Document(metadata={'source': 'D:/[24]ICT_Practice/practice_file/ex1.pdf', 'page': 0, 'start_index': 410}, page_content='* 쓰가루해협 등 일본 북쪽항로로 입항시 쓰가루해협 통과 후 부터 동경 135도30분 사이 또는')]


In [14]:
## 다양한 쿼리 생성

In [16]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

query = "일본 동부에서 평형수를 주입 후 국내 입항 예정인 선박은?"

llm = ChatOpenAI(temperature=0, model="gpt-4o", openai_api_key="개인키작성")

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

In [17]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [19]:
question = "일본 동부에서 평형수를 주입 후 국내 입항 예정인 선박은?"
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['일본 동부에서 평형수를 채운 후 한국으로 입항할 예정인 선박은?', '', '일본 동부에서 평형수를 주입한 후 한국에 도착할 예정인 배는?', '', '일본 동부에서 평형수를 채운 후 한국으로 들어오는 선박은?']


8

-> 쿼리 증폭..?

In [23]:
# 프롬프트 생성(Create Prompt)

In [24]:
from langchain import hub

In [25]:
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [27]:
# 최종 수행(?)

In [33]:
! pip install rank_bm25

In [34]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [41]:
# 단계 1: 문서 로드(Load Documents)
# 문서를 로드하고, 청크로 나누고, 인덱싱합니다.
from langchain.document_loaders import PyPDFLoader
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.schema import StrOutputParser

# PDF 파일 로드. 파일의 경로 입력
file_path = "D:/[24]ICT_Practice/practice_file/ex1.pdf"
loader = PyPDFLoader(file_path=file_path)

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

split_docs = loader.load_and_split(text_splitter=text_splitter)

# 단계 3, 4: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_docs, embedding=OpenAIEmbeddings(openai_api_key="개인키작성"))

# 단계 5: 리트리버 생성(Create Retriever)
# 사용자의 질문(query) 에 부합하는 문서를 검색합니다.

# 유사도 높은 K 개의 문서를 검색합니다.
k = 3

# (Sparse) bm25 retriever and (Dense) faiss retriever 를 초기화 합니다.
bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k = k

faiss_vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings(openai_api_key="개인키작성"))
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": k})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")

# 단계 7: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0,
                 openai_api_key="개인키작성")

def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 단계 8: 체인 생성(Create Chain)
rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke(question)

# 결과 출력
print(f"PDF Path: {file_path}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

PDF Path: D:/[24]ICT_Practice/practice_file/ex1.pdf
문서의 수: 1
[HUMAN]
일본 동부에서 평형수를 주입 후 국내 입항 예정인 선박은?

[AI]
일본 동부 6개현(아오모리현, 이와테현, 후쿠시마현, 미야기현, 이바라기현, 치바현)에서 평형수를 주입 후 국내 입항 예정인 선박입니다.


In [42]:
# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "평형수입항보고를 허위로 신고하거나 평형수관리기록부를 허위로 작성하면 어떻게 돼?"
response = rag_chain.invoke(question)
print(response)

평형수입항보고를 허위로 신고하거나 평형수관리기록부를 허위로 작성하는 경우, 「선박평형수 관리법」에 따라 항해정지(출항금지)나 과태료 처분을 받을 수 있습니다.


# csv 데이터 활용

In [61]:
#https://teddylee777.github.io/langchain/langchain-tutorial-04/

In [43]:
# OPENAI_API
import os

os.environ['OPENAI_API_KEY'] = "개인키작성"

In [53]:
import pandas as pd

# csv 파일을 데이터프레임으로 로드
df = pd.read_csv("D:/[24]ICT_Practice/practice_file/code.csv" , encoding='cp949')
df.head()

,순번,국가코드,국가한글명,국가영문명,국가대륙코드
0,1,AM,아르메니아,ARMENIA,아시아
1,2,AZ,아제르바이잔,AZERBAIJAN,아시아
2,3,BD,방글라데시,BANGLADESH,아시아
3,4,BN,브루나이,BRUNEI DARUSSALAM,아시아
4,5,BT,부탄,BHUTAN,아시아


In [58]:
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType


# 에이전트 생성
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, 
               model='gpt-4-0613',
               openai_api_key="개인키작성"),        # 모델 정의
    df,                                    # 데이터프레임
    verbose=True,                          # 추론과정 출력
    agent_type=AgentType.OPENAI_FUNCTIONS, # AgentType.ZERO_SHOT_REACT_DESCRIPTION
    allow_dangerous_code=True 
)

In [59]:
# 질의
agent.run('데이터의 행과 열의 갯수는 어떻게 돼?')



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape'}`


(249, 5)데이터프레임에는 총 249개의 행과 5개의 열이 있습니다.

> Finished chain.


'데이터프레임에는 총 249개의 행과 5개의 열이 있습니다.'

In [60]:
# 질의
agent.run('브루나인의 국가코드랑 국가대륙코드는 뭐야?')



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['국가한글명'] == '브루나이'][['국가코드', '국가대륙코드']]"}`


  국가코드 국가대륙코드
3   BN    아시아브루나이의 국가코드는 'BN'이고, 국가대륙코드는 '아시아'입니다.

> Finished chain.


"브루나이의 국가코드는 'BN'이고, 국가대륙코드는 '아시아'입니다."